In [2]:
from PIL import Image 
import random
from numpy import array 
import numpy as np

img = Image.open('../Lena.png')

ar = array(img)

In [3]:
ar

array([[160, 160, 160, ..., 169, 154, 129],
       [161, 160, 159, ..., 168, 154, 129],
       [160, 161, 160, ..., 168, 154, 129],
       ...,
       [ 50,  49,  55, ..., 106, 103, 104],
       [ 49,  50,  58, ..., 107, 109, 111],
       [ 50,  49,  59, ..., 107, 108, 112]], dtype=uint8)

In [4]:
ar2 = np.transpose(ar)

In [5]:
print(ar2)

[[160 161 160 ...  50  49  50]
 [160 160 161 ...  49  50  49]
 [160 159 160 ...  55  58  59]
 ...
 [169 168 168 ... 106 107 107]
 [154 154 154 ... 103 109 108]
 [129 129 129 ... 104 111 112]]


In [6]:
def shuffle_arr(arr):
    row = len(arr)
    col = len(arr[0])
    for i in range(0,row):
        for j in range(0,col):
            arr[i][j] = ((row*row)+i+1)%256
    for i in range(0,int(row/4)):
        for j in range(int(col/4)):
            arr[i][j] = (row*row + 1)%256 - arr[i][j]
    for i in range(0, int(row/4)):
        for j in range(3*int(col/3),col):
            arr[i][j] = ((row*row)+1)%256-arr[i][j]
    for i in range(3*(int(row/4)),row):
        for i in range(0,int(col/4)):
            arr[i][j] = (row*row+1)%256 - arr[i][j]
    for i in range(int(row/4),3*int(row/4)):
        for j in range(int(col/4),3*int(col/4)):
            arr[i][j] = (row*row+1)%256 -arr[i][j]
    
    temp = np.array(arr)

    for i in range(0,row):
        for j in range(0,col):
            rowindex = int((arr[i][j]-1)%row)
            colindex = int((arr[i][j]-1)/col)
            temp[i][j] = arr[rowindex][colindex]

    return temp

In [7]:
ar3 = shuffle_arr(ar2)

In [8]:
ar3

array([[  0,   0,   0, ...,   0,   0,   0],
       [255, 255, 255, ..., 255, 255, 255],
       [254, 254, 254, ..., 254, 254, 254],
       ...,
       [254, 254, 254, ..., 254, 254, 254],
       [255, 255, 255, ..., 255, 255, 255],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)

In [9]:
shuffled_img3 = Image.fromarray(ar3)

In [10]:
shuffled_img3.save('shuffeled_image3.png')

In [11]:
def key_gen(r,c):
    R = 3.99
    random_float = random.uniform(0,1)
    x = 0.4+(random_float/5)
    key = np.zeros((r,c))
    for i in range(0,r):
        for j in range(0,c):
            x = x*R*(1-x)
            key[i][j] = x*255
    
    return key

In [12]:
r = len(ar3)
c = len(ar3[0])
best_key = key_gen(r,c)
print(best_key)

[[248.31429786  25.9765497   93.08809319 ... 233.03403184  80.09444828
  219.19907854]
 [122.7907478  254.0154943    3.91301182 ... 196.78169066 179.25735837
  212.44678102]
 [141.45401816 251.31578884  14.48761855 ...  12.64426102  47.94899139
  155.34223233]
 ...
 [245.42401344  36.7733586  125.56649762 ... 233.22514296  79.4627143
  218.25564721]
 [125.48413088 254.2989146    2.78963988 ... 127.45887394 254.36247354
    2.537371  ]
 [ 10.0233706   38.42122065 130.20263557 ...  78.37756941 216.60594073
  130.12690783]]


In [13]:
def encrypt(ar3,key):
    r = len(ar3)
    c = len(ar3[0])
    for i in range(0,r):
        for j in range(0,c):
            ar3[i][j] = int(ar3[i][j]) ^ int(key[i][j])
            
    cipher_img = np.array(ar3)
    for itr in range(0,int(r/16)):
        for i in range(0,r):
            for j in range(0,c):
                xmap = int((i+j)%r)
                ymap = int((i+2*j)%c)
                cipher_img[i][j] = cipher_img[xmap][ymap]
                
    
    
    return ar3

In [14]:
encrypted_img_arr = encrypt(ar3,best_key)
print(encrypted_img_arr)

[[248  25  93 ... 233  80 219]
 [133   1 252 ...  59  76  43]
 [115   5 240 ... 242 209 101]
 ...
 [ 11 218 131 ...  23 177  36]
 [130   1 253 ... 128   1 253]
 [ 10  38 130 ...  78 216 130]]


In [15]:
encrypted_img = Image.fromarray(encrypted_img_arr)

In [16]:
encrypted_img.save('encrypted_img.png')

In [17]:
def correlation_coefficient(img1, img2):
    # Ensure that the images have the same shape
    assert img1.shape == img2.shape, "The images have different shapes"
    
    # Calculate the mean of the images
    mean1 = np.mean(img1)
    mean2 = np.mean(img2)
    
    # Calculate the standard deviation of the images
    std1 = np.std(img1)
    std2 = np.std(img2)
    
    # Subtract the mean from each pixel and divide by the standard deviation
    normalized_img1 = (img1 - mean1) / std1
    normalized_img2 = (img2 - mean2) / std2
    
    # Calculate the sum of the product of the corresponding pixels
    sum_of_products = np.sum(normalized_img1 * normalized_img2)
    
    # Calculate the correlation coefficient
    return sum_of_products / (img1.size - 1)

In [18]:
correlation_coefficient(ar,encrypted_img_arr)


0.0005462112046362731

In [19]:

def NPCR(img1, img2):
    """
    Calculate the NPCR value
    Bigger the better
    [0-100]
    Parameters:
        img1 = Encrypted image 1
        img2 = Encrypted image 2 with one pixel changed
    Returns:
        NPCR value
     """
    height, width = img1.shape
    sum = 0
    for i in range(height):
        for j in range(width):
            if img1[i, j] != img2[i, j]:
                sum += 1
    res = ((sum)/(width*height))*100
    return res


def UACI(img1, img2):
    """
    Calculate the UACI value
    Bigger the Better
    Parameters:
        img1 = Encrypted image 1
        img2 = Encrypted image 2 with one pixel changed
    Returns:
        UACI value
    """
    height, width = img1.shape
    difference = 0
    for y in range(height):
        for x in range(width):
            difference += (abs(int(img1[x, y])-int(img2[x, y])))
    difference = ((difference*100)/(width*height*255))
    return difference


def NPCR_UACI_worker(img1, img2):
    """
    Calculate the NPCR & UACI worker
    Parameters:
        img1 = Encrypted image 1
    Returns:
        NPCR value
    """
    return NPCR(img1, img2), UACI(img1, img2)


In [22]:
img2 = np.copy(ar3)
x = random.randint(0, 254)
y = random.randint(0, 254)
img2[x][y] = random.randint(0, 254)
img2 = encrypt(img2, best_key)

In [25]:
NPCR_UACI_worker(ar3,img2)

(100.0, 33.707600013882505)